# Adding transposed arrays with Dask and CuPy



## Use a DGX

In [ ]:
import dask
import dask.config

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

dask.config.set({"distributed.dashboard.link": "/proxy/8787/status"})
cluster = LocalCUDACluster(diagnostics_port=9100)
client = Client(cluster)
client

## Create Random Dataset

In [ ]:
import dask
import dask.array
import cupy


a = dask.array.random.random((1000, 1000), chunks=100)
a = a.persist()

## Convert data to GPU and persist in device memory

In [ ]:
import dask
import cupy

ga = a.map_blocks(cupy.asarray)
ga = ga.persist()

## Add array with its transpose

This takes an array and adds it to its transpose. This involves some computations that have dependencies and some that don't.

In [ ]:
out = ga + ga.T

In [ ]:
import time
start = time.time()

In [ ]:
out = out.persist()
%time _ = wait(out)

In [ ]:
out.shape

In [ ]:
_ = client.profile(start=start, filename='dask-cupy-add-transpose.html')

## Inspect output

In [ ]:
out[:10, :10].compute()

In [ ]:
from distributed.utils import format_bytes

In [ ]:
format_bytes(out.nbytes)